# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 4                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [1]:
!uv add -r requirements.txt --active

Resolved 208 packages in 12ms
Audited 200 packages in 0.55ms


## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [2]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

Overwriting strands_claude.py


#### Invoking local agent

In [3]:
!python strands_claude.py '{"prompt": "What is the weather now?"}'

I'll check the current weather for you.
Tool #1: weather
The current weather is sunny! It looks like a beautiful day outside.The current weather is sunny! It looks like a beautiful day outside.


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [4]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [5]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Navigate up to the utils.py location
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="strands_claude"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '78232eb3-2d3c-41c2-a4cd-3ac6332465ab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 19 Jul 2025 06:46:10 GMT', 'x-amzn-requestid': '78232eb3-2d3c-41c2-a4cd-3ac6332465ab', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-strands_claude-role
recreating agentcore-strands_claude-role
attaching role policy agentcore-strands_claude-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [6]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name+"2"
)
response

Bedrock AgentCore configured: /Users/youngjik/Documents/GitHub/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/youngjik/Documents/GitHub/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/youngjik/Documents/GitHub/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/Users/youngjik/Documents/GitHub/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-west-2', account_id='720025708473', execution_role='arn:aws:iam::720025708473:role/agentcore-strands_claude-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [7]:
launch_result = agentcore_runtime.launch()
launch_result

Build: #9 12.99 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [720025708473.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-strands_claude2]
8993cdb3c629: Preparing
c3d9bf5eadc2: Preparing
9f45f56a4187: Preparing
8498c9e4a8e2: Preparing
f1c21e71c58f: Preparing
14f24c28b3fe: Preparing
e47c67136f39: Preparing
83ab85380878: Preparing
58d7b7786e98: Preparing
14f24c28b3fe: Waiting
e47c67136f39: Waiting
83ab85380878: Waiting
58d7b7786e98: Waiting
8993cdb3c629: Pushed
f1c21e71c58f: Pushed
8498c9e4a8e2: Pushed
14f24c28b3fe: Pushed
83ab85380878: Pushed
c3d9bf5eadc2: Pushed
e47c67136f39: Pushed
58d7b7786e98: Pushed
9f45f56a4187: Pushed
latest: digest: sha256:b1ddb734ca076b19504645ae2a4fe4bfac8c66f3ae59cb3d76eca9a97491782f size: 2207


Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:720025708473:runtime/strands_claude2-GVEFVt6PIq
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/strands_claude2-GVEFVt6PIq-DEFAULT
   /aws/bedrock-agentcore/runtimes/strands_claude2-GVEFVt6PIq-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude2-GVEFVt6PIq-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude2-GVEFVt6PIq-DEFAULT --since 1h


LaunchResult(mode='cloud', tag='bedrock_agentcore-strands_claude2:latest', env_vars=None, port=None, runtime=None, ecr_uri='720025708473.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-strands_claude2', agent_id='strands_claude2-GVEFVt6PIq', agent_arn='arn:aws:bedrock-agentcore:us-west-2:720025708473:runtime/strands_claude2-GVEFVt6PIq', build_output=['#0 building with "default" instance using docker driver', '#1 [internal] load build definition from Dockerfile', '#1 transferring dockerfile: 786B done', '#1 DONE 0.0s', '#2 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim', '#2 DONE 2.2s', '#3 [internal] load .dockerignore', '#3 transferring context: 744B done', '#3 DONE 0.0s', '#4 [internal] load build context', '#4 transferring context: 1.22MB 0.0s done', '#4 DONE 0.0s', '#5 [1/6] FROM public.ecr.aws/docker/library/python:3.10-slim@sha256:9dd6774a1276178f94b0cc1fb1f0edd980825d0ea7634847af9940b1b6273c13', '#5 resolve public.ecr.aws/docker/library/python:3.10

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [12]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude2


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [8]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'f39a8051-79ae-427f-8067-7e35bcff615b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 19 Jul 2025 06:54:21 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f39a8051-79ae-427f-8067-7e35bcff615b',
   'baggage': 'Self=1-687b4118-226a34f445f7668e4a7390f1,session.id=8d89dedf-6398-4a72-9713-0e56f12d44f6',
   'x-amzn-bedrock-agentcore-runtime-session-id': '8d89dedf-6398-4a72-9713-0e56f12d44f6',
   'x-amzn-trace-id': 'Root=1-687b4118-354e0e0e4fd35c5a33094a5b;Self=1-687b4118-226a34f445f7668e4a7390f1'},
  'RetryAttempts': 0},
 'runtimeSessionId': '8d89dedf-6398-4a72-9713-0e56f12d44f6',
 'traceId': 'Root=1-687b4118-354e0e0e4fd35c5a33094a5b;Self=1-687b4118-226a34f445f7668e4a7390f1',
 'baggage': 'Self=1-687b4118-226a34f445f7668e4a7390f1,session.id=8d89dedf-6398-4a72-9713-0e56f12d44f6',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The weather

### Processing invocation results

We can now process our invocation results to include it in an application

In [9]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

The weather is currently sunny! It's a beautiful day outside.

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [17]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "1289x345는?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

1289 × 345 = **444,705**입니다.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [18]:
launch_result

LaunchResult(mode='cloud', tag='bedrock_agentcore-strands_claude2:latest', env_vars=None, port=None, runtime=None, ecr_uri='720025708473.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-strands_claude2', agent_id='strands_claude2-GVEFVt6PIq', agent_arn='arn:aws:bedrock-agentcore:us-west-2:720025708473:runtime/strands_claude2-GVEFVt6PIq', build_output=['#0 building with "default" instance using docker driver', '#1 [internal] load build definition from Dockerfile', '#1 transferring dockerfile: 786B done', '#1 DONE 0.0s', '#2 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim', '#2 DONE 2.2s', '#3 [internal] load .dockerignore', '#3 transferring context: 744B done', '#3 DONE 0.0s', '#4 [internal] load build context', '#4 transferring context: 1.22MB 0.0s done', '#4 DONE 0.0s', '#5 [1/6] FROM public.ecr.aws/docker/library/python:3.10-slim@sha256:9dd6774a1276178f94b0cc1fb1f0edd980825d0ea7634847af9940b1b6273c13', '#5 resolve public.ecr.aws/docker/library/python:3.10

In [19]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!